In [ ]:
pip install Flask


In [ ]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pickle

# Load the iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Save the model
with open('iris_model.pkl', 'wb') as f:
    pickle.dump(model, f)


In [ ]:
from flask import Flask, render_template, request, jsonify
import numpy as np
import pickle

app = Flask(__name__)

# Load the trained model
model = pickle.load(open('iris_model.pkl', 'rb'))

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    data = [float(x) for x in request.form.values()]
    features = [np.array(data)]
    prediction = model.predict(features)
    output = prediction[0]
    species = {0: 'setosa', 1: 'versicolor', 2: 'virginica'}

    return jsonify({'prediction': species[output]})

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Iris Flower Classification</title>
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css">
</head>
<body>
    <div class="container">
        <h1 class="mt-5">Iris Flower Classification</h1>
        <form id="predictionForm">
            <div class="form-group">
                <label for="sepal_length">Sepal Length</label>
                <input type="text" class="form-control" id="sepal_length" name="sepal_length">
            </div>
            <div class="form-group">
                <label for="sepal_width">Sepal Width</label>
                <input type="text" class="form-control" id="sepal_width" name="sepal_width">
            </div>
            <div class="form-group">
                <label for="petal_length">Petal Length</label>
                <input type="text" class="form-control" id="petal_length" name="petal_length">
            </div>
            <div class="form-group">
                <label for="petal_width">Petal Width</label>
                <input type="text" class="form-control" id="petal_width" name="petal_width">
            </div>
            <button type="submit" class="btn btn-primary">Predict</button>
        </form>
        <h3 class="mt-5" id="predictionResult"></h3>
    </div>

    <script src="https://code.jquery.com/jquery-3.3.1.min.js"></script>
    <script>
        $(document).ready(function() {
            $('#predictionForm').on('submit', function(event) {
                event.preventDefault();
                $.ajax({
                    url: '/predict',
                    method: 'POST',
                    data: $(this).serialize(),
                    success: function(response) {
                        $('#predictionResult').html('Predicted Species: ' + response.prediction);
                    },
                    error: function(error) {
                        console.log(error);
                    }
                });
            });
        });
    </script>
</body>
</html>


In [ ]:
python app.py
